# Hands-on Session for Customized Image Mining 1


Instructor: [Yuxiao (Rain) Luo](https://yuxiaoluo.github.io)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/YuxiaoLuo/AI_Intro/blob/main/week11_ImageMining_3_video.ipynb)

In this tutorial, we will detection the objects in a short video.


### Install necessary packages.

In [3]:
!pip install opencv-python opencv-python-headless opencv-contrib-python numpy matplotlib

Note: if this code returns an error such as 

```
""ERROR: Could not install packages due to an EnvironmentError: [WinError 5] Access is denied:
```
Try the code below.

In [ ]:
!!cmd
pip install --user opencv-python opencv-python-headless opencv-contrib-python numpy matplotlib

In [ ]:
pip install opencv-python opencv-python-headless opencv-contrib-python numpy matplotlib

### Make sure you have the 3 required files
You can download them from our GitHub repo: https://github.com/YuxiaoLuo/AI_Intro/tree/main/data/image_mining.

>Original sources:
>    - configuration file at https://gist.github.com/dkurt/54a8e8b51beb3bd3f770b79e56927bd7
>    - frozen_model file at https://github.com/datitran/object_detector_app/blob/master/object_detection/ssd_mobilenet_v1_coco_11_06_2017/frozen_inference_graph.pb
>    - cocoLabels file at https://github.com/amikelive/coco-labels/blob/master/coco-labels-2014_2017.txt


### Download the demo video ([traffic.mp4](https://github.com/YuxiaoLuo/AI_Intro/blob/main/data/customized_image_mining/traffic.mp4))
After you download it, please place the video in the folder of you Python script for object detection.

Video source: https://github.com/OlafenwaMoses/ImageAI/blob/master/data-videos/traffic.mp4

In [ ]:
# if you do not have such packages: opencv-python opencv-python-headless numpy matplotlib
# run the code below first
# pip install opencv-python opencv-python-headless numpy matplotlib


import cv2
import matplotlib.pyplot as plt
from google.colab.patches import cv2_imshow
    
# Load the model configuration and weights
config_file = 'ssd_mobilenet_v3_large_coco_2020_01_14.pbtxt'
frozen_model = 'frozen_inference_graph.pb'

model = cv2.dnn_DetectionModel(frozen_model, config_file)

# Load class labels
classLabels = []
file_name = 'cocoLabels.txt'
with open(file_name, 'rt') as fpt:
    classLabels = fpt.read().rstrip('\n').split('\n')
    
    
print(classLabels)

# Set model input parameters
model.setInputSize(320, 320)
model.setInputScale(1.0/127.5)
model.setInputMean((127.5, 127.5, 127.5))
model.setInputSwapRB(True)

# Open video source
cap = cv2.VideoCapture("traffic.mp4")  # Correct the function name

# Check if the video source is opened
if not cap.isOpened():
    cap = cv2.VideoCapture(0)
if not cap.isOpened():
    raise IOError("Cannot open video")
    
font_scale = 3
font = cv2.FONT_HERSHEY_PLAIN

# Object detection loop
while True:
    ret, frame = cap.read()
    
    if not ret:
        break
    
    # Assuming the model is defined earlier in code
    ClassIndex, confidence, bbox = model.detect(frame, confThreshold=0.55)
    
    if len(ClassIndex) != 0:
        for ClassInd, conf, boxes in zip(ClassIndex.flatten(), confidence.flatten(), bbox):
            if ClassInd <= 80:
                x, y, w, h = boxes
                cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 2)
                cv2.putText(frame, classLabels[ClassInd - 1], (x + 10, y + 40), font, 
                            fontScale=font_scale, color=(0, 255, 0), thickness=5)
    
    cv2_imshow(frame)
    
    if cv2.waitKey(2) & 0xFF == ord('q'):  # Exit on 'q' key press
        break

# Release resources
cap.release()
cv2.destroyAllWindows()